# Highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data

This notebook is primarily an intermediate to introducing to the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers..ipynb). Here the steps to see the difference between complexes represented in hu.MAP 2.0 and hu.MAP 3.0 data is done with one example. You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins. In ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb), it will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data. And that is more likely where you want to invest your time. You may want to quickly san the notebook below though to see the types of reports you'll expect when you give the snakemake file your list of identifiers. And with more details here it may help you troubleshoot such snakemake-generated reports. 

This effort just is meant to highlight possible differences. There is some approximating done to determine possible corresponding complexes that inherently brings in some possible judgement calls. You'll need to explore the results from each for yourself to compare in depth. The hu.MAP 2.0 data can be explored in sessions launched from my [humap2-binder repo](https://github.com/fomightez/humap2-binder).

------------

## Step #1: Preparation

The preparation parallels the notebooks ['Use of hu.MAP 3.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb) and ['Use of hu.MAP 2.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb), and so see them for more insight.

Get the data for hu.MAP 2.0 and hu.MAP 3.0:

In [1]:
!curl -OL https://raw.githubusercontent.com/fomightez/humap2-binder/refs/heads/main/additional_nbs/standardizing_initial_data/humap2_complexes_20200809InOrderMatched.csv
!curl -OL https://raw.githubusercontent.com/fomightez/humap3-binder/refs/heads/main/additional_nbs/standardizing_initial_data/hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  502k  100  502k    0     0  1870k      0 --:--:-- --:--:-- --:--:-- 1875k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1243k  100 1243k    0     0  2426k      0 --:--:-- --:--:-- --:--:-- 2424k


Get an accessory scripts for adding information about the proteins in the complexes & collecting differences between two versions:

In [2]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/make_lookup_table_for_extra_info4complexes.py
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/two_comp_three_details_plus_table.ipy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2893  100  2893    0     0  16392      0 --:--:-- --:--:-- --:--:-- 16344


##### Put the data on the complexes into Pandas dataframe

In [3]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/complexes_rawCSV_to_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1007  100  1007    0     0   3547      0 --:--:-- --:--:-- --:--:--  3558


In [2]:
!uv run complexes_rawCSV_to_df.py humap2_complexes_20200809InOrderMatched.csv
import pandas as pd
rd2_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
!uv run complexes_rawCSV_to_df.py hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv
rd3_df = pd.read_pickle('raw_complexes_pickled_df.pkl')

Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠹                                                                               Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               



--------

## Analyze complexes in hu.MAP 2.0 vs. hu.MAP 3.0 for a protein


In [3]:
search_term = "ROGDI"

Get complexes that protein is in in both sets of data, adding extra information like done in earlier notebooks.  
Then use that to make a summary table for change from version 2.0 to version 3.0.

In [ ]:
%run two_comp_three_details_plus_table.ipy

       HuMAP2_ID  Confidence Uniprot_ACCs genenames
6   HuMAP2_01834           3       Q9Y4E6      WDR7
7   HuMAP2_03388           2       Q9Y485     DMXL1
8   HuMAP2_03388           2       Q9GZN7     ROGDI
9   HuMAP2_03388           2       Q8TDJ6     DMXL2
10  HuMAP2_03388           2       Q9Y4E6      WDR7
 
          HuMAP3_ID  ComplexConfidence Uniprot_ACCs genenames
297  huMAP3_13872.1                  6       Q8NI08     NCOA7
298  huMAP3_13872.1                  6       Q8TDJ6     DMXL2
299  huMAP3_13872.1                  6       Q9GZN7     ROGDI
300  huMAP3_13872.1                  6       Q9Y485     DMXL1
301  huMAP3_13872.1                  6       Q9Y4E6      WDR7


That above should be a summary table quickly giving a sense of the improvement in 3.0 data. (Or lack thereof, if yo have substituted with your protein of interest?)

Despite general improvment, I noticed some information present in version 2.0 complexes went missing in complexes reported in version 3.0.
    
#### Check for disappearing proteins in complexes

In [9]:
# Make sure nothing present in HuMAP2 data disappeared in huMAP3 data
# Function to compare UniProt ACCs between HuMAP2 and HuMAP3
def check_disappearing_items(df_correspondences, df2_expanded, df3_expanded):
    # Tracking disappearing items
    disappearing_items = []
    
    for _, correspondence in df_correspondences.iterrows():
        humap2_id = correspondence['HuMAP2_ID']
        humap3_id = correspondence['HuMAP3_ID']
        
        # Get UniProt ACCs for this HuMAP2 group
        humap2_accs = set(df2_expanded[df2_expanded['HuMAP2_ID'] == humap2_id]['Uniprot_ACCs'])
        
        # Get UniProt ACCs for this HuMAP3 group
        humap3_accs = set(df3_expanded[df3_expanded['HuMAP3_ID'] == humap3_id]['Uniprot_ACCs'])
        
        # Find items in HuMAP2 that are not in HuMAP3
        lost_items = humap2_accs - humap3_accs
        
        if lost_items:
            disappearing_items.append({
                'HuMAP2_ID': humap2_id,
                'HuMAP3_ID': humap3_id,
                'lost_items': lost_items,
                'total_humap2_items': len(humap2_accs),
                'total_humap3_items': len(humap3_accs),
                'percent_lost': len(lost_items) / len(humap2_accs) * 100 if humap2_accs else 0
            })
    
    # Convert to DataFrame for easy analysis
    df_disappearing = pd.DataFrame(disappearing_items)
    
    # Display results
    global something_dropped_completely
    something_dropped_completely = False
    if not df_disappearing.empty:
        print("\nDisappearing Items between Hu.MAP2.0 and Hu.MAP3.0 data:")
        for _, row in df_disappearing.iterrows():
            print(f"\nHuMAP2 ID: {row['HuMAP2_ID']} -> HuMAP3 ID: {row['HuMAP3_ID']}")
            print(f"Total items in HuMAP2: {row['total_humap2_items']}")
            print(f"Total items in HuMAP3: {row['total_humap3_items']}")
            print(f"Percent of items lost: {row['percent_lost']:.2f}%")
            print("UniProt ACCs Lost From Corresponding Complex:")
            for item in row['lost_items']:
                dropped_completely = (item not in df3_expanded['Uniprot_ACCs'].to_list())
                if dropped_completely:
                    print(f"  - {item} (AND NOT IN ANY OTHER Hu.MAP3.0 {search_term}-complexes!!!)")
                    something_dropped_completely = True
                else:
                    print(f"  - {item} (But present in other, {search_term}-related Hu.MAP3.0 complexes)")
        
        # Optional: Save to CSV
        df_disappearing.to_csv('disappearing_items.csv', index=False)
        
        return df_disappearing, something_dropped_completely
    else:
        print("\nNo disappearing items found between HuMAP2 and HuMAP3.")
        return None, something_dropped_completely

# Call the function
disappearing_df, something_dropped_completely = check_disappearing_items(df_correspondences, df2_expanded, df3_expanded)


Disappearing Items between Hu.MAP2.0 and Hu.MAP3.0 data:

HuMAP2 ID: HuMAP2_01834 -> HuMAP3 ID: huMAP3_12042.1
Total items in HuMAP2: 5
Total items in HuMAP3: 56
Percent of items lost: 20.00%
UniProt ACCs Lost From Corresponding Complex:
  - P50993 (AND NOT IN ANY OTHER Hu.MAP3.0 ROGDI-complexes!!!)

HuMAP2 ID: HuMAP2_03388 -> HuMAP3 ID: huMAP3_10511.1
Total items in HuMAP2: 4
Total items in HuMAP3: 35
Percent of items lost: 25.00%
UniProt ACCs Lost From Corresponding Complex:
  - Q8TDJ6 (But present in other, ROGDI-related Hu.MAP3.0 complexes)


In [10]:
# TO DO: Also make sure nothing that is in the majority of complexes in hu.MAP 2.0 data disappears entirely from hu.MAP 3.0 data.
# That can only occur as the case if `something_dropped_completely`, that we've already check on, is True
majority_item_dropped_completely = False #set up by declaring false
if something_dropped_completely:
    # collect information on what identifiers occur in the majority of complexes in hu.MAP 2.0 data
    # the search term should always be present and can thus serve as a sanity check for this collection
    majority_identifiers = []
    def find_majority_uniprot_accs(df):
        # Group by HuMAP2_ID and get unique Uniprot_ACCs for each complex
        complex_accs = df.groupby('HuMAP2_ID')['Uniprot_ACCs'].unique()
        
        # Count total number of unique complexes
        total_complexes = len(complex_accs)
        
        # Flatten and count occurrences of each Uniprot ACC across complexes
        all_accs = df['Uniprot_ACCs'].tolist()
        acc_counts = pd.Series(all_accs).value_counts()
        
        # Find ACCs that appear in more than 50% of complexes
        majority_accs = acc_counts[acc_counts >= 0.51 * total_complexes].index.tolist()
        
        return majority_accs
    if find_majority_uniprot_accs(df2_expanded):
        majority_identifiers = find_majority_uniprot_accs(df2_expanded)
    # Now check none of those `majority_identifiers` match the disappearing ones dropped completely
    print("\nHu.MAP2.0-Majority Complex Members Disappearing in Hu.MAP3.0 data:")
    for item in list(set.union(*disappearing_df['lost_items'])): # note `list(set.union(*disappearing_df['lost_items']))` gets the unique set members from the 'lost_items' column
        if item in majority_identifiers:
            majority_item_dropped_completely = (item not in df3_expanded['Uniprot_ACCs'].to_list())
            if majority_item_dropped_completely:
                print(f"  - {item} IS IN MAJORITY OF Hu.MAP2.0 COMPLEXES YET NOT IN ANY Hu.MAP3.0 {search_term}-complexes!!! (PERHAPS CONCERNING?)")
                majority_item_dropped_completely = True
    if majority_item_dropped_completely == False:
        print("NONE")


Hu.MAP2.0-Majority Complex Members Disappearing in Hu.MAP3.0 data:
NONE


 You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins, and so you'll want to first check out the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb). That notebook will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data.
 
-----

Enjoy!

See my [humap3-binder repo](https://github.com/fomightez/humap3-binder) and [humap3-utilities](https://github.com/fomightez/structurework/humap3-utilities) for related information & resources for this notebook.



-----
